# When Slowness Problems Get Complex

## 1. Parallelizing Operations

We've called that a few times already, that reading information from disk or transferring it over the network is a slow operation. In typical scripts while this operation is going on, nothing else happens. The script is blocked, waiting for input or output while the CPU sits idle. 

One way we can make this better is to do operations in parallel. That way, while the computer is waiting for the slow IO, other work can take place. The tricky part is dividing up the tasks so that we get the same result in the end. There's actually a whole field of computer science called **concurrency,** dedicated to how we write programs that do operations in parallel. We won't go into a ton of details here but we'll give you a brief overview of what you can do. 

First, we need to understand what the operating system already does for us. Our OS handles the many processes that run on our computer. If a computer has more than one core, the operating system can decide which processes get executed on which core, and no matter the split between cores, all of these processes will be executing in parallel. Each of them has its own memory allocation and does its own IO calls. The OS will decide what fraction of CPU time each process gets and switch between them as needed. 

So a very easy way to run operations in parallel is just to split them across different processes, calling your script many times each with a different input set, and just let the operating system handle the concurrency. Let's say you want to collect statistics on the current load and memory usage for all the computers in your network. You can do this by writing a script that connects to each computer in a list and gets the stats. Each connection takes a while to complete, so the total run-time of the script would be the sum of the time taken by all those connections. Instead, you could split the list of computers into smaller groups and use the OS to call the script many times once for each group. That way, the connections to the different computers can be started in parallel, which minimizes the time but the CPU isn't doing anything. 

This is super easy to do and for many scripts, it'll be the right choice. Another easy thing to do, is to have a good balance of different workloads that you run on a computer. If you have a process that's using a lot of CPU while a different process is using a lot of network IO and another process is using a lot of disk IO, these can all run in parallel without interfering with each other. 

When using the OS to split the work and the processes, these processes don't share any memory, and sometimes we might need to have some shared data. In that case, we'd use threads. **Threads** *let us run parallel tasks inside a process.* This allows threats to share some of the memory with other threads in the same process. Since this isn't handled by the OS, we'll need to modify our code to create and handle the threads. For that, we'll need to look into how the programming language we're using implements threading. In Python, we can use the **Threading** or **AsyncIO** modules to do this. These modules let us specify which parts of the code we want to run in separate threads or as separate asynchronous events, and how we want the results of each to be combined in the end. We won't go into the details of how you can do that here, but we'll link to more information on this in the next reading. 

One thing to watch out for is that depending on the actual threading implementation for the language you're using, it might happen that all threads get executed in the same CPU processor. In that case, if you want to use more processors, you'll need to split the code into fully separate processes. If your script is mostly just waiting on input or output, also known as I/O bound, it not might matter if it's executed on one processor or eight. 

But you might be doing this in parallel because you're using all of the available CPU time. In other words, your script is 
**CPU bound.** In this case, you'll definitely want to split your execution across processors. Now there's a point where adding more parallel processes means things become even slower, not faster. If we're trying to read a bunch of files from disk and do too many operations in parallel, the disk might end up spending more time going from one position to another then actually retrieving the data, or if we're doing a ton of operations that use a lot of CPU, the OS could spend more time switching between them than actually making progress in the calculations we're trying to do. 

So when doing operations in parallel, we need to find the right balance of simultaneous actions that let our computers stay busy without starving our system for resources. 

I recently felt the benefits of applying concurrency. I was working on migrating data that was stored in one format, and I needed to store it in a different format. There were a lot of gigabytes of data that needed migrating, so of course I wasn't going to do it manually. My first version of the script was taking an average of one hour per gigabyte migrated. This was much slower than I expected, so I decided to spend more time tweaking the code to make the migration go faster. I reorganized the logic to have a separate thread per file which decreased the total time to work through the files since it now wasn't a linear process, and then, to make it go even faster, I split the work onto different machines, each running a bunch of threads. After all this rearranging to use the resources I have, I brought it down to three minutes per gigabyte. Yeah, we may be learning a lot about slowness, but we sure are moving fast. Some of these concepts may feel a little complex and it's perfectly normal. Take your time, everyone learns slowness at their own pace. Up next, we'll discuss the different approaches we can take to handle systems as they become more complex.

## 2. Slowly Growing in Complexity

As we called out in an earlier video, a solution that's good for one problem might not be so good for a different problem. And as a system becomes more complex and grows in usage, a solution that worked well before may no longer be well-suited. Let's say you're writing a secret Santa script where each person gives a secret gift to one other randomly assigned person. The script randomly selects pairs of people and then sends an email to the gift-giver telling them who they're buying a present for. If you're doing this for the people working on your floor, you might just store the list of names and emails in a CSV file.

The file will be small enough that the time spent parsing it won't be significant. Now if this script grows into a larger project that handles everyone at your company and the company keeps hiring more and more people, at some point parsing the file will start taking a lot of time. This is where you might want to consider using a different technology.

For example, you could decide to store your data in a **SQLite** file. This is a lightweight database system that lets you query the information stored in the file without needing to run a database server. Using SQLite for the data probably works just fine for assigning secret Santas at your company. 

But imagine that you've kept adding features to the service. So it now includes a way to create a wish list, a machine learning algorithm that suggests possible gifts and a tracker that keeps a history of each present given. And since people at your company love the program so much, you've made it an external service available to anybody. Keeping all the data in one file would be too slow. So you'll need to move to a different solution. You have to use a fully-fledged database server. Probably even running on a separate machine than the one running the secret Santa service. And there's even one more step after that. If the service becomes really really popular, you might notice that your database isn't fast enough to serve all the queries being requested. In that case, you can add a caching service like memcached which keeps the most commonly used results in RAM to avoid querying the database unnecessarily. So we've gone from hosting the data in a CSV file to having it in a SQLite file then moving it to a database server and finally using a dynamic casher in front of the database server to make it run even faster.

A similar progression can happen on the user facing side of the same project. Initially, we set the Santa service would simply send emails to the people on the list. That's fine if it's a small group and there's one person in charge of the script. But as the project grows more complex, you'd want to have a website for the service to let people do things like check who their assigned person is and create wish lists. Initially, this could just be running on a web server on the same machine as the data. If the website gets used a lot, you might need to add a caching service like Varnish. This would speed up the load of dynamically created pages. And eventually, this still might not be enough. So you need to distribute your service across many different computers and use a load balancer to distribute the requests. You could do this in-house with separate computers hosted at your company, but this means that as the application keeps growing you need to add more and more servers. It might be easier to use virtual machines running in the cloud that can be added or removed as the load sustained by the service changes.

These examples show how important it is to find the right solution for each problem. It makes no sense to deploy a multi server web service with a distributed database for storage when you're only going to have a few dozen users. You just need to pay attention to how the service is growing to know when you need to take the next step to make it work best for the current use case. Up next, we'll talk about some of the issues that you might come across when dealing with complex systems.

## 3. Dealing with Complex Slow Systems

In our last video, we discussed how systems that grow in usage also grow in complexity. In large complex systems, we have lots of different computers involved. Each one doing a part of the work and interacting with the others through the network. 

For example, think of an e-commerce site for your company. The web server is the part of the system that directly interacts with external users. Another component is the database server, which is accessed by the code that handles any requests generated from the website, and depending on how the whole system is built, you might have a bunch of other services involved doing different parts of the work. There could be a billing system that generates invoices once orders are placed. A fulfillment system used by the employees preparing the orders for customers. A reporting system that once a day creates a report of all the sales placed and possibly more. On top of this, you should probably have backup, monitoring, testing infrastructure, and so on. 

A system like this can be tricky to debug and understand. What do you do if your complex system is slow? As usual, what you want to do is find the bottleneck that's causing your infrastructure to underperform. Is it the generation of dynamic pages on the web server? Is it the queries to the database? Is it doing the calculations for the fulfillment process? 

Figuring this out can be tricky. So one key piece is to have a good monitoring infrastructure that lets you know where the system is spending the most time. Saying notice that getting the web pages is pretty slow. But when you check the web server, you see that it's not overloaded. Instead, most of the time is spent waiting on network calls, and when looking at your database server, you find that it's spending a lot of time on Disk I/O. This shows that there's a problem with how the data is being accessed in the database. 

One thing to look at is the indexes present in the database. When a database server needs to find data, it can do it much faster if there's an index on the field that you're querying for. On the flip side, if the database has too many indexes, adding or modifying entries can become really slow because all of the indexes need updating. 

So we need to look for a good balance of having indexes for the fields that are actually going to be used. If the problem is not solved by indexing and there are too many queries for the server to reply to all of them on time, you might need to look into either caching the queries or distributing the data to separate database servers. 

Now what if when you try to figure out why the service is slow, you see that the CPU on the web serving machine is saturated. The first step is to check if the code of the service can be improved using the techniques that we explained earlier. If it's a dynamic website, we might try adding caching on top of it. But if the code is fine and the cache doesn't help because the problem is that there's just too many requests coming in for one machine to answer all of them, you'll need to distribute the load across more computers. 

To make this possible, you might need to reorganize the code so that it's capable of running in a distributed system instead of on a single computer. This might take some work, but once you've done it, you can easily scale your application to as many requests as needed by adding more computers to the system, and finally, make sure that you actually need to do whatever you're doing. Lots of times, as projects evolve, we're left with a scary monster of layer after layer of complex code. If we think about what our system is doing for a few minutes, we might end up discovering that there's a whole piece that wasn't needed at all and it was making our servers do unnecessary work all along.

If all of this is starting to sound too difficult and scary, don't worry. Remember that if you ever need to deal with such complex systems, one of your best tools is to ask your colleagues for help. Up next, we'll try our hand at solving a real life problem with something complex.